In [ ]:
#Collap Drive Bağlama
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Gerekli Kütüphaneler
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
import joblib
from sklearn.metrics import classification_report

In [ ]:
#Train ve Test Veri Setleri Kurulumu

path = "/content/drive/MyDrive/Binovist  Sentiment Analysis/data/Hepsiburada Dataset 492k(%10 test)/"

df_train = pd.read_csv(path + "train.csv")
df_test = pd.read_csv(path + "test.csv")

dataframes = {'df_train': df_train, 'df_test': df_test}

yes_responses = {'evet', 'yes', 'y', '1', 'tamam', 'onay', 'olur', 'e'}
no_responses = {'hayır', 'no', 'n', '0', 'hayir', 'olmaz', 'h'}

In [ ]:
df_train_filtered = pd.read_csv(path + "df_train_filtered.csv")
df_test_filtered = pd.read_csv(path + "df_test_filtered.csv")

dataframes_filtered = {'df_train': df_train_filtered, 'df_test': df_test_filtered}

#Eğer Temizleme Sonrası Boş Veri Olursa Çıkarma İşlemi
for name, df in dataframes_filtered.items():
  dataframes_filtered[name] = df.dropna()

In [ ]:
# Oluştururalan yeni temiz verinin veri setine ekstra bir kolonlar'ı
df_train_filtered = dataframes_filtered['df_train']
df_test_filtered = dataframes_filtered['df_test']

X_train=df_train_filtered["processed_text"]
X_test=df_test_filtered["processed_text"]
y_train=df_train_filtered["label_numeric"]
y_test=df_test_filtered["label_numeric"]

print("x_train",X_train.shape)
print("x_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)

x_train (152609,)
x_test (48951,)
y_train (152609,)
y_test (48951,)


In [ ]:
print(X_train.isnull().sum())
print(X_test.isnull().sum())
print(y_train.isnull().sum())
print(y_test.isnull().sum())

0
0
0
0


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC())
])

In [ ]:
parameters = {
    'tfidf__max_df': [0.9, 1.0],
    'tfidf__min_df': [1, 2],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'svm__C': [0.1, 1, 10],
    'svm__kernel': ['linear', 'rbf']
}

grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=2)

In [ ]:
# Hiperparametre optimizasyonu yapmadan eğitim
pipeline.fit(X_train, y_train)

# Eğer GridSearchCV kullanıyorsanız:
# grid_search.fit(X_train, y_train)
# print("En iyi parametreler: ", grid_search.best_params_)

# Test seti üzerinde tahmin yapma
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.87      0.74      5655
           1       0.94      0.98      0.96     17087
           2       0.97      0.87      0.92     26209

    accuracy                           0.91     48951
   macro avg       0.85      0.91      0.87     48951
weighted avg       0.92      0.91      0.91     48951

